In [ ]:

import os
'''
openai==0.28.1
import openai
openai.api_type = "azure"
openai.api_base = ""

openai.api_key = ""
openai.api_version = '2023-12-01-preview' # This API version or later is required to access fine-tuning for turbo/babbage-002/davinci-002
'''
from openai import AzureOpenAI
client = AzureOpenAI(
  azure_endpoint = "", 
  api_key="",  
  api_version="2023-12-01-preview"  # This API version or later is required to access fine-tuning for turbo/babbage-002/davinci-002
)
training_file_name = './finetune_data/soccer_training_set.jsonl' #or training_set.jsonl

# Upload the training and validation dataset files to Azure OpenAI with the SDK.

'''
openai==0.28.1
training_response = openai.File.create(
    file=open(training_file_name, "rb"), purpose="fine-tune", user_provided_filename="soccer_training_set.jsonl"
)
training_file_id = training_response["id"]
'''

training_response = client.files.create(
    file=open(training_file_name, "rb"), purpose="fine-tune"
)

training_file_id = training_response.id

print("Training file ID:", training_file_id)




In [ ]:
'''
openai==0.28.1
response = openai.FineTuningJob.create(
    training_file=training_file_id,
    model="gpt-35-turbo-0613",
)

job_id = response["id"]

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.

print("Job ID:", response["id"])
print("Status:", response["status"])
print(response)
'''
response = client.fine_tuning.jobs.create(
    training_file=training_file_id,
    model="gpt-35-turbo-0613" # Enter base model name. Note that in Azure OpenAI the model name contains dashes and cannot contain dot/period characters. 
)

job_id = response.id

# You can use the job ID to monitor the status of the fine-tuning job.
# The fine-tuning job will take some time to start and complete.

print("Job ID:", response.id)
print("Status:", response.id)
print(response.model_dump_json(indent=2))

In [ ]:
from IPython.display import clear_output
import time

start_time = time.time()

# Get the status of our fine-tuning job.
#response = openai.FineTuningJob.retrieve(job_id)
response = client.fine_tuning.jobs.retrieve(job_id)
status = response.status

# If the job isn't done yet, poll it every 10 seconds.
while status not in ["succeeded", "failed"]:
    time.sleep(10)
    
    #response = openai.FineTuningJob.retrieve(job_id)
    response = client.fine_tuning.jobs.retrieve(job_id)
    print(response)
    print("Elapsed time: {} minutes {} seconds".format(int((time.time() - start_time) // 60), int((time.time() - start_time) % 60)))
    status = response.status
    print(f'Status: {status}')
    clear_output(wait=True)

print(f'Fine-tuning job {job_id} finished with status: {status}')

# List all fine-tuning jobs for this resource.
print('Checking other fine-tune jobs for this resource.')
#response = openai.FineTuningJob.list()
response = client.fine_tuning.jobs.list()
print(f'Found {len(response.data)} fine-tune jobs.')

In [ ]:
#response = openai.FineTuningJob.retrieve(job_id)
response = client.fine_tuning.jobs.retrieve(job_id)
print(response)
fine_tuned_model = response.status